In [ ]:
#Setting up the data collection : import and API connexion variables:

import pandas as pd
import numpy as np
import requests
import json
import os
import bcrypt
import datetime
import pickle
from getpass import getpass
import time
from IPython.display import display

password = os.getenv('SORAREpwd')
api_key = os.getenv('SORARE_APIKEY')
salt = b"$2a$11$KnNxrgfQuLaxCfahjb3z3." 

encoded_password = password.encode('utf-8')
hashed_password = bcrypt.hashpw(encoded_password, salt)
hashed_password_string = hashed_password.decode()

In [ ]:
email = getpass("login: ")

In [ ]:
#Sign-in to API (1st Step)

sign_in_initial_url = "https://api.sorare.com/graphql"

# Define the headers for the initial sign-in request
headers = {
    "Content-Type": "application/json"
}

# Construct the payload for the initial sign-in mutation
sign_in_initial_payload = {
    "operationName": "SignInMutation",
    "variables": {
        "input": {
            "email": email,
            "password": hashed_password_string
        }
    },
    "query": """
        mutation SignInMutation($input: signInInput!) {
            signIn(input: $input) {
                otpSessionChallenge
                errors {
                    message
                }
            }
        }
    """
}

# Make the initial sign-in request
sign_in_initial_response = requests.post(sign_in_initial_url, headers=headers, data=json.dumps(sign_in_initial_payload))
sign_in_initial_json_response = sign_in_initial_response.json()

# Handle the initial sign-in response
otp_session_challenge = sign_in_initial_json_response["data"]["signIn"]["otpSessionChallenge"]

In [ ]:
otp_2FA = getpass('2FA:')

In [ ]:
# Sign-in with 2FA request
sign_in_2fa_url = "https://api.sorare.com/federation/graphql"

# Construct the payload for the sign-in with 2FA mutation
sign_in_2fa_payload = {
    "operationName": "SignInMutation",
    "variables": {
        "input": {
            "otpSessionChallenge": otp_session_challenge,
            "otpAttempt": otp_2FA
        }
    },
    "query": """
        mutation SignInMutation($input: signInInput!) {
            signIn(input: $input) {
                currentUser {
                    slug
                    jwtToken(aud: "<YourAud>") {
                        token
                        expiredAt
                    }
                }
                errors {
                    message
                }
            }
        }
    """
}

# Make the sign-in with 2FA request
sign_in_2fa_response = requests.post(sign_in_2fa_url, headers=headers, data=json.dumps(sign_in_2fa_payload))
sign_in_2fa_json_response = sign_in_2fa_response.json()

In [ ]:
user_key = sign_in_2fa_json_response['data']['signIn']['currentUser']['jwtToken']['token']

In [ ]:
#Function for Querying Sorare API from Python. Input is the GraphQL Query to put in it

def graphql_request(query):
    # Replace with your API URL
    url = "https://api.sorare.com/federation/graphql"

    # Define the headers for the request
    headers = {
        "Content-Type": "application/json",
        "APIKEY": api_key
    }

    # Create the payload with the query
    payload = {
        "query": query
    }

    # Make the GraphQL request
    response = requests.post(url, headers=headers, json=payload)

    # Parse the JSON response
    json_data = response.json()

    # Return the JSON response
    return json_data

In [ ]:
# Query to fetch all cards from Clubs (target is to loop that query on a list of all clubs from a League)

cardsbyclubs_query = '''
    query getAllLimitedCardsByClubs {
  allCards(
    rarities: [limited]
    teamSlugs: ["real-madrid-madrid"]) {
    nodes {
      id
      assetId
      name
      slug
      rarity
      createdAt
      seasonYear
      grade
      power
      xp
      serialNumber
      shirtNumber
      customCardEditionName
      player {
        slug
        activeClub {
          name
        }
      }
      ownershipHistory {
        id
        token {
          id
          ethereumId
        }
        from
        transferType
        priceWei
        priceFiat {
          eur
        }
        
        user {
          id
          slug
        }
      }
    }
    pageInfo {
      endCursor
    }
  }
}
  '''

In [ ]:
test = graphql_request(cardsbyclubs_query)
test

In [ ]:
test['data']['allCards']['pageInfo']['endCursor']

In [ ]:
# Functions For Looping Queries: 

def build_query_first50(team):
    query = '''
    query getAllLimitedCardsByClubs {
  allCards(
    rarities: [limited]
    teamSlugs: "%s"
  ) {
    nodes {
      id
      assetId
      name
      slug
      rarity
      createdAt
      seasonYear
      grade
      power
      xp
      serialNumber
      shirtNumber
      customCardEditionName
      player {
        slug
        activeClub {
          name
        }
      }
      ownershipHistory {
        from
        transferType
        priceWei
        priceFiat {
          eur
        }
        user {
          id
          slug
        }
      }
    }
    pageInfo {
      endCursor
    }
  }
}
  ''' % team
    
    return query
    
def build_query_next50(cursor,team):
    query = '''
    query getAllLimitedCardsByClubs {
  allCards(
    rarities: [limited]
    teamSlugs: "%s"
    after: "%s"
  ) {
    nodes {
      id
      assetId
      name
      slug
      rarity
      createdAt
      seasonYear
      grade
      power
      xp
      serialNumber
      shirtNumber
      customCardEditionName
      player {
        slug
        activeClub {
          name
        }
      }
      ownershipHistory {
        from
        transferType
        priceWei
        priceFiat {
          eur
        }
        user {
          id
          slug
        }
      }
    }
    pageInfo {
      endCursor
    }
  }
}
  ''' % (team,cursor)
    return query


In [ ]:
# Function for looping in order to get all cards (API Paginated per 50 results)

def get_all_cards(team):
    responses = []
    query = build_query_first50(team)
    response = graphql_request(query)
    responses.append(response)
    end_cursor = response['data']['allCards']['pageInfo']['endCursor']
    start_cursor = end_cursor
    x=0
    while start_cursor:
        query = build_query_next50(start_cursor,team)
        response = graphql_request(query)
        responses.append(response)
        end_cursor = response['data']['allCards']['pageInfo']['endCursor']
        start_cursor = end_cursor
        if x%10 == 0: 
            print('x =',x, end_cursor, datetime.datetime.now())
            with open('card_records_backup.pickle', 'wb') as handle:
                pickle.dump(responses, handle, protocol=pickle.HIGHEST_PROTOCOL)
        x+=1
        if x==2000:
            break
    return responses



In [ ]:
'''
list_of_teams = ["real-madrid-madrid",
                 "girona-girona",
                 "barcelona-barcelona", 
                 "atletico-madrid-madrid",
                 "athletic-club-bilbao",
                 "real-sociedad-donostia-san-sebastian", 
                 "valencia-valencia", 
                 "real-betis-sevilla",
                 "las-palmas-las-palmas-de-gran-canaria",
                "getafe-getafe-madrid",
                 "deportivo-alaves-vitoria-gasteiz",
                 "osasuna-pamplona-irunea",
                 "rayo-vallecano-madrid",
                 "villarreal-villarreal",
                 "sevilla-sevilla-1890",
                 "celta-de-vigo-vigo",
                 "mallorca-palma-de-mallorca",
                 "cadiz-cadiz",
                 "granada-granada",
                 "almeria-almeria",
                 "real-valladolid-valladolid",
                 "espanyol-barcelona",
                 "elche-elche",
                 "levante-valencia"
                ]
all_cards_per_team = []
for team in list_of_teams:
    all_cards_per_team.append(get_all_cards(team))
    with open('team_card_records_backup.pickle', 'wb') as handle:
                pickle.dump(all_cards_per_team, handle, protocol=pickle.HIGHEST_PROTOCOL)

                '''

In [ ]:
all_cards_per_team

In [ ]:

with open('team_card_records_backup.pickle', 'rb') as handle:
    team_all_card_pickled = pickle.load(handle)


In [ ]:
total_nodes = 0

for card_list in team_all_card_pickled:
    for card in card_list:
        nodes = card['data']['allCards']['nodes']
        total_nodes += len(nodes)

print(total_nodes)

In [ ]:
nodes = all_cards[0]['data']['allCards']['nodes']

In [ ]:


card_data = []
for card in team_all_card_pickled[0]['data']['allCards']['nodes']:
    for history in card.get('ownershipHistory', []):
        card_info = {
            'card_id': card['id'],
            'asset_id': card['assetId'],
            'card_name': card['name'],
            'season_year': card['seasonYear'],
            'card_slug': card['slug'],
            'rarity': card['rarity'],
            'card_creation_date': card['createdAt'],
            'grade': card['grade'],
            'power': card['power'],
            'xp': card['xp'],
            'serial_number': card['serialNumber'],
            'shirt_number': card['shirtNumber'],
            'special_edition': card['customCardEditionName'],
            'transaction_date': history['from'],
            'transaction_type': history['transferType'],
            'price_wei': history['priceWei'],
            'price_eur': history['priceFiat']['eur'],
            'user_id':history['user']['id'],
            'user_slug':history['user']['slug']
        }
        player_info = card.get('player', {}).get('activeClub')
        if player_info:
            card_info.update(player_info)
        card_data.append(card_info)

df = pd.DataFrame(card_data)
display(df)

In [ ]:
df['card_name'].nunique()

In [ ]:
card_data = []
list_of_clubs = ["real-madrid-madrid", "girona-girona", "barcelona-barcelona", "atletico-madrid-madrid", "athletic-club-bilbao", "real-sociedad-donostia-san-sebastian", "valencia-valencia", "real-betis-sevilla", "las-palmas-las-palmas-de-gran-canaria", "getafe-getafe-madrid", "deportivo-alaves-vitoria-gasteiz", "osasuna-pamplona-irunea", "rayo-vallecano-madrid", "villarreal-villarreal", "sevilla-sevilla-1890", "celta-de-vigo-vigo", "mallorca-palma-de-mallorca", "cadiz-cadiz", "granada-granada", "almeria-almeria", "real-valladolid-valladolid", "espanyol-barcelona", "elche-elche", "levante-valencia"]

for i, card_list in enumerate(team_all_card_pickled):
    team_name = list_of_clubs[i]
    
    for card_dict in card_list:
        nodes = card_dict['data']['allCards']['nodes']
        
        for card in nodes:
            for history in card.get('ownershipHistory', []):
                if history and history.get('user') is not None:  
                    card_info = {
                        'team_name': team_name,
                        'card_id': card['id'],
                        'asset_id': card['assetId'],
                        'card_name': card['name'],
                        'season_year': card['seasonYear'],
                        'card_slug': card['slug'],
                        'rarity': card['rarity'],
                        'card_creation_date': card['createdAt'],
                        'grade': card['grade'],
                        'power': card['power'],
                        'xp': card['xp'],
                        'serial_number': card['serialNumber'],
                        'shirt_number': card['shirtNumber'],
                        'special_edition': card['customCardEditionName'],
                        'transaction_date': history['from'],
                        'transaction_type': history['transferType'],
                        'token_id': history['token']['id'],
                        'price_wei': history['priceWei'],
                        'price_eur': history['priceFiat']['eur'],
                        'user_id': history['user']['id'],
                        'user_slug': history['user']['slug'],
                        'player_slug': None  # Add a default value for 'player_slug'
                    }
                    player_info = card.get('player')
                    if player_info and player_info.get('activeClub') and player_info.get('slug'):
                        card_info['player_slug'] = player_info['slug']
                    card_data.append(card_info)

df = pd.DataFrame(card_data)


In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.to_csv('all_cards_transactions_la_liga.csv', index=False)

In [ ]:
df['transaction_type'].value_counts()

In [ ]:
df['user_slug'].value_counts()

In [ ]:
df[df['user_slug'] == 'pitmen'].sort_values(by='transaction_date', ascending=True)


In [ ]:
df[df['card_slug'] == 'jordi-alba-ramos-2021-limited-164']

In [ ]:
#test query for fetching a player scores and stats:


player_all_scores = '''
query PlayerAllScores {
  football {
    player(slug: "%s") {
      ...PlayerFields
      allSo5Scores(after: "") {
        nodes {
          id
          scoringVersion
          score
          decisiveScore {
            totalScore
          }
          game {
            date
            so5Fixture {
              gameWeek
              id
              slug
            }
          }
        }
        pageInfo {
          endCursor
        }
      }
    }
  }
}

fragment PlayerFields on Player {
  firstName
  lastName
  displayName
  slug
  activeClub {
    name
    shortName
    slug
    id
    activeCompetitions {
      id
      name
    }
    id
    dataPartner {
      name
    }
  }
  bestFoot
  cardPositions
  cardSupply {
    season {
      id
      name
      startYear
    }
    limited
  }
  careerSeasons {
    name
    id
  }
  injuries {
    active
    id
    kind
    startDate
    expectedEndDate
  }
  suspensions {
    active
    competition {
      name
      id
    }
    startDate
    kind
    matches
    reason
  }
  age
  stats2021: stats(seasonStartYear: 2021) {
    season {
      name
      id
      startYear
    }
    appearances
    assists
    goals
    id
    minutesPlayed
    redCards
    substituteIn
    substituteOut
    yellowCards
  }
  stats2022: stats(seasonStartYear: 2022) {
    season {
      name
      id
      startYear
    }
    appearances
    assists
    goals
    id
    minutesPlayed
    redCards
    substituteIn
    substituteOut
    yellowCards
  }
  stats2023: stats(seasonStartYear: 2023) {
    season {
      name
      id
      startYear
    }
    appearances
    assists
    goals
    id
    minutesPlayed
    redCards
    substituteIn
    substituteOut
    yellowCards
  }
  u23Eligible
  u23EligibleUntil
}
''' 


In [ ]:
test_sc = graphql_request(player_all_scores)
test_sc

In [ ]:
test_sc['data']['football']['player']['allSo5Scores']['pageInfo']['endCursor']

In [ ]:
def scores_query_first50(player):
    query = '''
    query PlayerAllScores {{
      football {{
        player(slug: "{player}") {{
          ...PlayerFields
          allSo5Scores(after: "") {{
            nodes {{
              id
              scoringVersion
              score
              decisiveScore {{
                totalScore
              }}
              game {{
                date
                so5Fixture {{
                  gameWeek
                  id
                  slug
                }}
              }}
            }}
            pageInfo {{
              endCursor
            }}
          }}
        }}
      }}
    }}
    
    fragment PlayerFields on Player {{
      firstName
      lastName
      displayName
      slug
      id
      activeClub {{
        name
        shortName
        slug
        id
        activeCompetitions {{
          id
          name
        }}
        id
        dataPartner {{
          name
        }}
      }}
      bestFoot
      cardPositions
      cardSupply {{
        season {{
          id
          name
          startYear
        }}
        limited
      }}
      careerSeasons {{
        name
        id
      }}
      injuries {{
        active
        id
        kind
        startDate
        expectedEndDate
      }}
      suspensions {{
        active
        competition {{
          name
          id
        }}
        startDate
        kind
        matches
        reason
      }}
      age
      stats2021: stats(seasonStartYear: 2021) {{
        season {{
          name
          id
          startYear
        }}
        appearances
        assists
        goals
        id
        minutesPlayed
        redCards
        substituteIn
        substituteOut
        yellowCards
      }}
      stats2022: stats(seasonStartYear: 2022) {{
        season {{
          name
          id
          startYear
        }}
        appearances
        assists
        goals
        id
        minutesPlayed
        redCards
        substituteIn
        substituteOut
        yellowCards
      }}
      stats2023: stats(seasonStartYear: 2023) {{
        season {{
          name
          id
          startYear
        }}
        appearances
        assists
        goals
        id
        minutesPlayed
        redCards
        substituteIn
        substituteOut
        yellowCards
      }}
      u23Eligible
      u23EligibleUntil
    }}
    '''.format(player=player) 
    return query
    
def scores_query_next50(player, cursor):
    query = '''
    query PlayerAllScores {{
      football {{
        player(slug: "{player}") {{
          ...PlayerFields
          allSo5Scores(after: "{cursor}") {{
            nodes {{
              id
              scoringVersion
              score
              decisiveScore {{
                totalScore
              }}
              game {{
                date
                so5Fixture {{
                  gameWeek
                  id
                  slug
                }}
              }}
            }}
            pageInfo {{
              endCursor
            }}
          }}
        }}
      }}
    }}
    
    fragment PlayerFields on Player {{
      firstName
      lastName
      displayName
      slug
      id
      activeClub {{
        name
        shortName
        slug
        id
        activeCompetitions {{
          id
          name
        }}
        id
        dataPartner {{
          name
        }}
      }}
      bestFoot
      cardPositions
      cardSupply {{
        season {{
          id
          name
          startYear
        }}
        limited
      }}
      careerSeasons {{
        name
        id
      }}
      injuries {{
        active
        id
        kind
        startDate
        expectedEndDate
      }}
      suspensions {{
        active
        competition {{
          name
          id
        }}
        startDate
        kind
        matches
        reason
      }}
      age
      stats2021: stats(seasonStartYear: 2021) {{
        season {{
          name
          id
          startYear
        }}
        appearances
        assists
        goals
        id
        minutesPlayed
        redCards
        substituteIn
        substituteOut
        yellowCards
      }}
      stats2022: stats(seasonStartYear: 2022) {{
        season {{
          name
          id
          startYear
        }}
        appearances
        assists
        goals
        id
        minutesPlayed
        redCards
        substituteIn
        substituteOut
        yellowCards
      }}
      stats2023: stats(seasonStartYear: 2023) {{
        season {{
          name
          id
          startYear
        }}
        appearances
        assists
        goals
        id
        minutesPlayed
        redCards
        substituteIn
        substituteOut
        yellowCards
      }}
      u23Eligible
      u23EligibleUntil
    }}
    '''.format(player=player, cursor=cursor)
    return query

In [ ]:
def get_all_scores(list_of_players):
    scores_data = {}
    
    for index, player in enumerate(list_of_players):
        scores_data[index] = []
        try:
            time.sleep(0.1)
            print("new_player")
            query = scores_query_first50(player)
            response = graphql_request(query)
            scores_data[index].append(response)
            
            if response is None:
                print(f"Error fetching data for player {index}: Response is None")
                continue
            
            end_cursor = response['data']['football']['player']['allSo5Scores']['pageInfo']['endCursor']
            start_cursor = end_cursor

            while start_cursor:
                time.sleep(0.1)
                query = scores_query_next50(player, start_cursor)
                response = graphql_request(query)
                
                if response is None:
                    print(f"Error fetching data for player {index}: Response is None")
                    break

                scores_data[index].append(response)

                end_cursor = response['data']['football']['player']['allSo5Scores']['pageInfo']['endCursor']
                start_cursor = end_cursor
        except KeyError as e:
            print(f"Error fetching data for player {index}: Missing key {str(e)}")
            break

    return scores_data


In [ ]:
list_of_players_test = ['paulo-dino-gazzaniga','toni-fuidias-ribera','aleix-garcia-serrano']

In [ ]:
'''OLD WORKING STUFF:

def create_player_dataframe(player_scores, list_of_players):
    player_data = []
    
    for index, player in enumerate(list_of_players):
        for score_data in player_scores[index]:
            player_info = score_data['data']['football']['player']
            
            player_name = player_info['displayName']
            player_slug = player_info['slug']
            player_id = player_info['id']
            age = player_info['age']
            u23_eligible = player_info['u23Eligible']
            u23_until = player_info['u23EligibleUntil']
            
            all_so5_scores = player_info['allSo5Scores']['nodes']
            
            for game_week_data in all_so5_scores:
                so5_score = game_week_data['score']
                decisive_score = game_week_data['decisiveScore']['totalScore']
                
                game_data = game_week_data.get('game')  # Use .get() method to handle missing keys
                
                if game_data is not None and game_data['so5Fixture'] is not None:
                    gameweek_date = game_data['date']
                    gameweek_number = game_data['so5Fixture'].get('gameWeek')  # Use .get() method to handle missing keys
                    gameweek_id = game_data['so5Fixture'].get('id')  # Use .get() method to handle missing keys
                    
                    player_data.append({
                        'player_name': player_name,
                        'player_slug': player_slug,
                        'player_id': player_id,
                        'age': age,
                        'u23_eligible': u23_eligible,
                        'u23_until': u23_until,
                        'so5_score': so5_score,
                        'decisive_score': decisive_score,
                        'gameweek_date': gameweek_date,
                        'gameweek_number': gameweek_number,
                        'gameweek_id': gameweek_id
                    })
    
    df = pd.DataFrame(player_data)
    
    # Calculate rolling average of last 5 scores excluding null/0 scores
    df['rolling_avg_last_5_scores'] = df.groupby('player_id')['so5_score'].apply(lambda x: x.rolling(5, min_periods=1).mean().mask(x.eq(0), other=np.nan)).reset_index(0, drop=True)
    
    # Calculate rolling average of last 15 scores excluding null/0 scores
    df['rolling_avg_last_15_scores'] = df.groupby('player_id')['so5_score'].apply(lambda x: x.rolling(15, min_periods=1).mean().mask(x.eq(0), other=np.nan)).reset_index(0, drop=True)
    
    return df'''

In [ ]:
def create_player_dataframe(player_scores, list_of_players):
    player_data = []

    for player_scores_list in player_scores.values():
        for score_data in player_scores_list:
            player_info = score_data['data']['football']['player']
            player_name = player_info['displayName']
            player_slug = player_info['slug']
            player_id = player_info['id']
            age = player_info['age']
            u23_eligible = player_info['u23Eligible']
            u23_until = player_info['u23EligibleUntil']
            all_so5_scores = player_info['allSo5Scores']['nodes']

            for game_week_data in all_so5_scores:
                so5_score = game_week_data['score']
                decisive_score = game_week_data['decisiveScore']['totalScore']
                game_data = game_week_data.get('game')

                if game_data is not None and game_data['so5Fixture'] is not None:
                    gameweek_date = game_data['date']
                    gameweek_number = game_data['so5Fixture'].get('gameWeek')
                    gameweek_id = game_data['so5Fixture'].get('id')

                    player_data.append({
                        'player_name': player_name,
                        'player_slug': player_slug,
                        'player_id': player_id,
                        'age': age,
                        'u23_eligible': u23_eligible,
                        'u23_until': u23_until,
                        'so5_score': so5_score,
                        'decisive_score': decisive_score,
                        'gameweek_date': gameweek_date,
                        'gameweek_number': gameweek_number,
                        'gameweek_id': gameweek_id
                    })

    df = pd.DataFrame(player_data)

    return df

In [ ]:
#test2_player_scores = get_all_scores(list_of_players_test)

In [ ]:
test2_player_scores

In [ ]:
test2_player_df = create_player_dataframe(test2_player_scores,list_of_players_test)

In [ ]:
test2_player_df

In [ ]:
import matplotlib.pyplot as plt

def plot_scores_by_gameweek(df):
    # Generate a unique color for each player
    unique_players = df['player_id'].unique()
    num_players = len(unique_players)
    color_map = plt.get_cmap('tab10')
    color_list = color_map(np.linspace(0, 1, num_players))
    
    # Plot the scores for each player by gameweek
    plt.figure(figsize=(10, 6))
    
    for player in unique_players:
        player_df = df[df['player_id'] == player]
        plt.scatter(player_df['gameweek_number'], player_df['so5_score'], c=[color_list[i] for i in range(num_players) if unique_players[i] == player], label=player_df['player_name'].iloc[0])
    
    plt.xlabel('Gameweek Number')
    plt.ylabel('SO5 Score')
    plt.title('SO5 Scores by Gameweek')
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
plot_scores_by_gameweek(test2_player_df)

In [ ]:
la_liga_cards_df=pd.read_csv('all_cards_transactions_la_liga.csv')

In [ ]:
la_liga_cards_df.head()

In [ ]:
la_liga_cards_df['player_slug'].isna().sum()

In [ ]:
la_liga_cards_df.dropna(subset=['player_slug'], inplace=True)

In [ ]:
la_liga_players_list = la_liga_cards_df['player_slug'].unique().tolist()

In [ ]:
len(la_liga_cards_df)

In [ ]:
len(la_liga_players_list)

In [ ]:
print(la_liga_players_list)

In [ ]:
with open('liga_players.pickle', 'wb') as file:
    pickle.dump(la_liga_players_scores, file)

In [ ]:
#la_liga_players_df = create_player_dataframe(la_liga_players_scores,la_liga_players_list)

In [ ]:
la_liga_players_df = create_player_dataframe(la_liga_players_scores,la_liga_players_list)

In [ ]:
list_of_players_set = set(la_liga_players_list)
len(list_of_players_set)

In [ ]:
la_liga_players_scores_set = set(la_liga_players_scores['player_slug'])
len(la_liga_players_scores_set)

In [ ]:

missing_players = list_of_players_set - la_liga_players_scores_set
missing_players

In [ ]:
len(la_liga_players_scores)

In [ ]:
len(la_liga_players_list)

In [ ]:
print(type(la_liga_players_scores))
print(type(la_liga_players_list))

In [ ]:
la_liga_players_scores

In [ ]:
#la_liga_players_scores.to_csv('la_liga_players_scores.csv', index=False)

In [ ]:
# Step 1: Group the data by 'team_name'
groups = la_liga_cards_df.groupby('team_name')

# Step 2-4: Generate sub-dataframes and unique player slug lists for each club
club_player_lists = []
for team_name, group_df in groups:
    sub_df = group_df.copy()  # Create a copy of the subgroup
    player_slugs = sub_df['player_slug'].unique().tolist()
    club_player_lists.append((team_name, player_slugs))

In [ ]:
# Iterate over the list of tuples
for team, player_list in club_player_lists:
    if team == "real-madrid-madrid":
        real_madrid_players = player_list
        break  # Exit the loop after finding the desired club

# Accessing the player list for Real Madrid
print(real_madrid_players)

In [ ]:
len(real_madrid_players)

In [ ]:
real_madrid_players_scores